In [116]:
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

def read_all_data(table_name):
    select_records = "SELECT * from " + table_name
    records = execute_read_query(connection, select_records)

    for rec in records:
        print(rec)

connection = create_connection("./data/sqliteDB")

# Завдання 1
# Зробіть таблицю для підрахунку особистих витрат із такими полями: id, призначення, сума, час.
create_table = """
CREATE TABLE IF NOT EXISTS spendings (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  purpose TEXT NOT NULL,
  total REAL NOT NULL,
  time TIMESTAMP NOT NULL
);
"""
execute_query(connection, create_table)

Connection to SQLite DB successful
Query executed successfully


In [48]:
# Завдання 2
# Створіть консольний інтерфейс (CLI) на Python для додавання нових записів до бази даних.
import datetime

print('Options:\n1:Show all records;2:Add a new record;-1:Exit')
while True:
    c = input('Choose option:')
    if c == '-1':
        break
    elif c == '1':
        select_spendings = "SELECT * from spendings"
        spendings = execute_read_query(connection, select_spendings)

        for spend in spendings:
            print(spend)
    elif c == '2':
        purpose = input('Purpose:')
        total = float(input('Total:'))
        time = datetime.datetime.now()
        insert_spend = """
            INSERT INTO
                spendings (purpose, total, time)
            VALUES
        """ + f"""("{purpose}", {total}, "{time}")"""
         
        execute_query(connection, insert_spend)

Options:
1:Show all records;2:Add a new record;-1:Exit


Choose option: 2
Purpose: test 1
Total: 23


Query executed successfully


Choose option: 2
Purpose: test 2
Total: 45


Query executed successfully


Choose option: 1


(1, 'test 1', 23.0, '2023-03-30 18:06:54.100396')
(2, 'test 2', 45.0, '2023-03-30 18:07:00.672055')


Choose option: -1


In [49]:
# Завдання 3
# Змініть таблицю так, щоби можна було додати не лише витрати, а й прибутки.
drop_table = "DROP TABLE IF EXISTS transactions;"
execute_query(connection, drop_table)
rename_table = "ALTER TABLE spendings RENAME TO transactions;"
execute_query(connection, rename_table)
add_column = "ALTER TABLE transactions ADD COLUMN is_benefit BOOLEAN;"
execute_query(connection, add_column)

Query executed successfully
Query executed successfully
Query executed successfully


In [50]:
read_all_data('transactions')

(1, 'test 1', 23.0, '2023-03-30 18:06:54.100396', None)
(2, 'test 2', 45.0, '2023-03-30 18:07:00.672055', None)


In [51]:
update_query = """UPDATE transactions set is_benefit = false"""
execute_query(connection, update_query)
read_all_data('transactions')

Query executed successfully
(1, 'test 1', 23.0, '2023-03-30 18:06:54.100396', 0)
(2, 'test 2', 45.0, '2023-03-30 18:07:00.672055', 0)


In [72]:
# Завдання 4
# Створіть агрегатні функції для підрахунку загальної кількості  витрат i прибуткiв за місяць. Забезпечте відповідний інтерфейс користувача.
import datetime
from dateutil.relativedelta import relativedelta

print('Options:\n1:Show all records;2:Add a new record;3:Month revenue;4: Month losses;-1:Exit')
while True:
    c = input('Choose option:')
    if c == '-1':
        break
    elif c == '1':
        select_spendings = "SELECT * from transactions"
        spendings = execute_read_query(connection, select_spendings)

        for spend in spendings:
            print(spend)
    elif c == '2':
        purpose = input('Purpose:')
        total = float(input('Total:'))
        time = datetime.datetime.now()
        s = input('Is benefit (Y/N):')
        is_benefit = True if (s == 'y' or s == 'Y') else False
        insert_spend = """
            INSERT INTO
                transactions (purpose, total, time, is_benefit)
            VALUES
        """ + f"""("{purpose}", {total}, "{time}", {is_benefit})"""
         
        execute_query(connection, insert_spend)
    elif c == '3' or c == '4':
        currentDateTime = datetime.datetime.now()
        month_ago = currentDateTime - relativedelta(months=1)
        is_benefit = 1 if c == '3' else 0
        total = f"""SELECT SUM(total) from transactions 
                            WHERE is_benefit = {is_benefit} AND 
                            time > '{month_ago}' AND time < '{currentDateTime}';
                        """
        total = execute_read_query(connection, total)
        print("Total:", total[0][0])
    

Options:
1:Show all records;2:Add a new record;3:Month revenue;4: Month losses;-1:Exit


Choose option: 2
Purpose: test 3
Total: 234
Is benefit (Y/N): Y


Query executed successfully


Choose option: 1


(1, 'test 1', 23.0, '2023-03-30 18:06:54.100396', 0)
(2, 'test 2', 45.0, '2023-03-30 18:07:00.672055', 0)
(3, 'test 3', 234.0, '2023-03-30 18:15:55.238005', 1)


Choose option: 3


Total: 234.0


Choose option: 4


Total: 68.0


Choose option: -1


In [75]:
# Завдання 5
# Create an Exchange Rates To USD db using API Monobank (api.monobank.ua). Do requests via request lib, parse results, write it into db. (3 examples required)
# Example:
# Table - Exchange Rate To USD:
# id (INT PRIMARY KEY) - 1, 2, 3, ...
# currency_name (TEXT) - UAH
# currency_value (REAL) - 39.5
# current_date (DATETIME) - 10/22/2022 7:00 PM
create_table = """
CREATE TABLE IF NOT EXISTS exchange_rate_usd (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  currency_name TEXT NOT NULL,
  currency_value REAL NOT NULL,
  current_date TIMESTAMP NOT NULL
);
"""
execute_query(connection, create_table)

Query executed successfully


In [86]:
datetime.datetime.now()

datetime.datetime(2023, 3, 30, 18, 49, 8, 929519)

In [76]:
read_all_data('exchange_rate_usd')

In [90]:
pd.to_datetime(int('1680186607'), utc=True, unit='s')

Timestamp('2023-03-30 14:30:07+0000', tz='UTC')

In [93]:
import requests
import pandas as pd
url = 'https://api.monobank.ua/bank/currency'
response = requests.get(url)
data = response.json()
currency_codes = {'840': 'USD', '978': 'EUR', '980': 'UAH', '826': 'GBP'}
data[0]

{'currencyCodeA': 840,
 'currencyCodeB': 980,
 'date': 1680127274,
 'rateBuy': 36.65,
 'rateCross': 0,
 'rateSell': 37.4406}

In [112]:
def get_currency(data):
    rate_col = ''
    if data['currencyCodeA'] == 840:
        currency_name = data['currencyCodeB']
        rate_col = 'rateSell'
    elif data['currencyCodeB'] == 840:
        currency_name = data['currencyCodeA']
        rate_col = 'rateBuy'
    else:
        return
    currency_name = currency_codes[str(currency_name)]
    currency_value = data[rate_col]
    current_date = pd.to_datetime(int(data['date']), utc=True, unit='s')
    return currency_name, currency_value, current_date
records = []
for d in data:
    res = get_currency(d)
    if res != None:
        records.append(res)
print(records)   

records = [f"""('{currency_name}', {currency_value}, '{current_date}')""" for currency_name, currency_value, current_date in records]

[('UAH', 37.4406, Timestamp('2023-03-29 22:01:14+0000', tz='UTC')), ('EUR', 1.08, Timestamp('2023-03-30 14:30:07+0000', tz='UTC'))]


In [113]:
insert_query = """
    INSERT INTO
        exchange_rate_usd (currency_name, currency_value, current_date)
    VALUES
""" + ",\n".join(list(records)) + ";"
print(insert_query)
execute_query(connection, insert_query)


    INSERT INTO
        exchange_rate_usd (currency_name, currency_value, current_date)
    VALUES
('UAH', 37.4406, '2023-03-29 22:01:14+00:00'),
('EUR', 1.08, '2023-03-30 14:30:07+00:00');
Query executed successfully


In [114]:
read_all_data('exchange_rate_usd')

(1, 'UAH', 37.4406, '2023-03-29 22:01:14+00:00')
(2, 'EUR', 1.08, '2023-03-30 14:30:07+00:00')


In [115]:
connection.close()